# Run CGD on multiple datasets, with various parameters to compute accuracy evolution.

In [1]:
lapply(c("dplyr","Seurat","HGNChelper", "adverSCarial", "IRdisplay",
         "ComplexHeatmap","RColorBrewer","circlize", "ggplot2", "stringr"), library, character.only = T)
options(warn=-1)


Attaching package: ‘dplyr’


The following objects are masked from ‘package:stats’:

    filter, lag


The following objects are masked from ‘package:base’:

    intersect, setdiff, setequal, union


Loading required package: SeuratObject

Loading required package: sp

The legacy packages maptools, rgdal, and rgeos, underpinning the sp package,
which was just loaded, will retire in October 2023.
Please refer to R-spatial evolution reports for details, especially
https://r-spatial.org/r/2023/05/15/evolution4.html.
It may be desirable to make the sf package available;
package maintainers should consider adding sf to Suggests:.
The sp package is now running under evolution status 2
     (status 2 uses the sf package in place of rgdal)


Attaching package: ‘SeuratObject’


The following objects are masked from ‘package:base’:

    intersect, t


Loading required package: grid

ComplexHeatmap version 2.18.0
Bioconductor page: http://bioconductor.org/packages/ComplexHeatmap/
Github page: ht

[[1]]
[1] "dplyr"     "stats"     "graphics"  "grDevices" "utils"     "datasets" 
[7] "methods"   "base"     

[[2]]
 [1] "Seurat"       "SeuratObject" "sp"           "dplyr"        "stats"       
 [6] "graphics"     "grDevices"    "utils"        "datasets"     "methods"     
[11] "base"        

[[3]]
 [1] "HGNChelper"   "Seurat"       "SeuratObject" "sp"           "dplyr"       
 [6] "stats"        "graphics"     "grDevices"    "utils"        "datasets"    
[11] "methods"      "base"        

[[4]]
 [1] "adverSCarial" "HGNChelper"   "Seurat"       "SeuratObject" "sp"          
 [6] "dplyr"        "stats"        "graphics"     "grDevices"    "utils"       
[11] "datasets"     "methods"      "base"        

[[5]]
 [1] "IRdisplay"    "adverSCarial" "HGNChelper"   "Seurat"       "SeuratObject"
 [6] "sp"           "dplyr"        "stats"        "graphics"     "grDevices"   
[11] "utils"        "datasets"     "methods"      "base"        

[[6]]
 [1] "ComplexHeatmap" "grid"           "IRdisplay"      "adverSCarial"  
 [5] "HGNChelper"     "Seurat"         "SeuratObject"   "sp"            
 [9] "dplyr"          "stats"          "graphics"       "grDevices"     
[13] "utils"          "datasets"       "methods"        "base"          

[[7]]
 [1] "RColorBrewer"   "ComplexHeatmap" "grid"           "IRdisplay"     
 [5] "adverSCarial"   "HGNChelper"     "Seurat"         "SeuratObject"  
 [9] "sp"             "dplyr"          "stats"          "graphics"      
[13] "grDevices"      "utils"          "datasets"       "methods"       
[17] "base"          

[[8]]
 [1] "circlize"       "RColorBrewer"   "ComplexHeatmap" "grid"          
 [5] "IRdisplay"      "adverSCarial"   "HGNChelper"     "Seurat"        
 [9] "SeuratObject"   "sp"             "dplyr"          "stats"         
[13] "graphics"       "grDevices"      "utils"          "datasets"      
[17] "methods"        "base"          

[[9]]
 [1] "ggplot2"        "circlize"       "RColorBrewer"   "ComplexHeatmap"
 [5] "grid"           "IRdisplay"      "adverSCarial"   "HGNChelper"    
 [9] "Seurat"         "SeuratObject"   "sp"             "dplyr"         
[13] "stats"          "graphics"       "grDevices"      "utils"         
[17] "datasets"       "methods"        "base"          

[[10]]
 [1] "stringr"        "ggplot2"        "circlize"       "RColorBrewer"  
 [5] "ComplexHeatmap" "grid"           "IRdisplay"      "adverSCarial"  
 [9] "HGNChelper"     "Seurat"         "SeuratObject"   "sp"            
[13] "dplyr"          "stats"          "graphics"       "grDevices"     
[17] "utils"          "datasets"       "methods"        "base"

In [2]:
bd <- "data/"

source("load_classifiers_v5.r")

In [3]:
# Datasets to test
c_basen = c("hgnc_axilla_10k", "hgnc_kidney_10k", "hgnc_liver_10k")
c_b2 = c("pbmc_3k", "axilla_10k", "kidney_10k", "liver_10k")
c_ds4exp <- list(pbmc_3k="", kidney_10k="kidney_10k__", liver_10k="liver_10k__", axilla_10k="axilla_10k__")
# Classifiers to test
c_strClass <- c("RF", "scType", "scAnnotatR", "MLP")
# Parameters to test
c_epsilon_alpha = c(0.5,1,2)
# Chosen cell type clusters in each dataset
chosenClusters <- list(axilla_10k=c("blood vessel endothelial cell"),
                      kidney_10k=c("kidney cell"),
                      liver_10k=c("endothelial cell"))
# Datasets path
c_paths <- list(pbmc_3k=list(so=paste0(bd, "v2//pbmc_test.rds"),
                             sce=paste0(bd, "v2//sce_pbmc_test.rds"),
                             df=paste0(bd, "v2//seurat_scaled_pbmc_test.txt")),
                kidney_10k=list(so=paste0(bd, "v5/data/sc/hgnc_kidney_10k_test.rds"),
                               sce=paste0(bd, "v5/data/sc/sce_hgnc_kidney_10k_test.rds"),
                               df=paste0(bd, "v5/data/sc/hgnc_kidney_10k_test.txt")),
                liver_10k=list(so=paste0(bd, "v5/data/sc/hgnc_liver_10k_test.rds"),
                               sce=paste0(bd, "v5/data/sc/sce_hgnc_liver_10k_test.rds"),
                               df=paste0(bd, "v5/data/sc/hgnc_liver_10k_test.txt")),
                axilla_10k=list(so=paste0(bd, "v5/data/sc/hgnc_axilla_10k_test.rds"),
                               sce=paste0(bd, "v5/data/sc/sce_hgnc_axilla_10k_test.rds"),
                               df=paste0(bd, "v5/data/sc/hgnc_axilla_10k_test.txt"))
               )

# Build cell type classif correspondance

In [4]:
type_corres <- readRDS("data//v5/type_corres.rds")

In [5]:
names(type_corres)

[1] "scType__pbmc_3k__Memory CD4 T"                                          
  [2] "scType__pbmc_3k__B"                                                     
  [3] "scType__pbmc_3k__CD14+ Mono"                                            
  [4] "scType__pbmc_3k__NK"                                                    
  [5] "scType__pbmc_3k__CD8 T"                                                 
  [6] "scType__pbmc_3k__Naive CD4 T"                                           
  [7] "scType__pbmc_3k__FCGR3A+ Mono"                                          
  [8] "scType__pbmc_3k__DC"                                                    
  [9] "scType__pbmc_3k__Platelet"                                              
 [10] "scType__liver_10k__malignant cell"                                      
 [11] "scType__liver_10k__endothelial cell"                                    
 [12] "scType__liver_10k__blood vessel smooth muscle cell"                     
 [13] "scType__liver_10k__macrophage"                                          
 [14] "scType__liver_10k__monocyte"                                            
 [15] "scType__liver_10k__mature NK T cell"                                    
 [16] "scType__kidney_10k__kidney epithelial cell"                             
 [17] "scType__kidney_10k__kidney cell"                                        
 [18] "scType__kidney_10k__podocyte"                                           
 [19] "scType__kidney_10k__epithelial cell of proximal tubule"                 
 [20] "scType__kidney_10k__mesenchymal cell"                                   
 [21] "scType__kidney_10k__kidney loop of Henle epithelial cell"               
 [22] "scType__axilla_10k__malignant cell"                                     
 [23] "scType__axilla_10k__blood vessel endothelial cell"                      
 [24] "scType__axilla_10k__fibroblast"                                         
 [25] "scType__axilla_10k__macrophage"                                         
 [26] "scType__axilla_10k__blood vessel smooth muscle cell"                    
 [27] "scType__axilla_10k__T cell"                                             
 [28] "MLP__pbmc_3k__Memory CD4 T"                                             
 [29] "MLP__pbmc_3k__B"                                                        
 [30] "MLP__pbmc_3k__CD14+ Mono"                                               
 [31] "MLP__pbmc_3k__NK"                                                       
 [32] "MLP__pbmc_3k__CD8 T"                                                    
 [33] "MLP__pbmc_3k__Naive CD4 T"                                              
 [34] "MLP__pbmc_3k__FCGR3A+ Mono"                                             
 [35] "MLP__pbmc_3k__DC"                                                       
 [36] "MLP__pbmc_3k__Platelet"                                                 
 [37] "MLP_liver_10k__liver_10k__malignant cell"                               
 [38] "MLP_liver_10k__liver_10k__endothelial cell"                             
 [39] "MLP_liver_10k__liver_10k__blood vessel smooth muscle cell"              
 [40] "MLP_liver_10k__liver_10k__macrophage"                                   
 [41] "MLP_liver_10k__liver_10k__monocyte"                                     
 [42] "MLP_liver_10k__liver_10k__mature NK T cell"                             
 [43] "MLP_kidney_10k__kidney_10k__kidney epithelial cell"                     
 [44] "MLP_kidney_10k__kidney_10k__kidney cell"                                
 [45] "MLP_kidney_10k__kidney_10k__podocyte"                                   
 [46] "MLP_kidney_10k__kidney_10k__epithelial cell of proximal tubule"         
 [47] "MLP_kidney_10k__kidney_10k__mesenchymal cell"                           
 [48] "MLP_kidney_10k__kidney_10k__kidney loop of Henle epithelial cell"       
 [49] "MLP_axilla_10k__axilla_10k__malignant cell"                             
 [50] "MLP_axilla_10k__axilla_10k__blood vessel endothelial cell"              
 [

In [6]:
lsg <- list(pbmc_3k= read.table("data//v2/sign_genes.txt")$gene,
           liver_10k = read.table("data//v5/data/sign/sign_liver_10k.txt")$gene,
           kidney_10k = read.table("data//v5/data/sign/sign_kidney_10k.txt")$gene,
           axilla_10k = read.table("data//v5/data/sign/sign_axilla_10k.txt")$gene)

In [7]:
exportResults <- function(acbc, baseDir, expID, strClassifier){
    write.table(acbc$byGeneSummary,
        paste0(baseDir,"/byGeneSummary_",expID), row.names=F)
    display(acbc$oneRowSummary)
    dfOneRowSummary <- as.data.frame(matrix(nrow=0, ncol=17))
    dfOneRowSummary <- rbind(dfOneRowSummary, c(strClassifier, acbc$oneRowSummary))
    colnames(dfOneRowSummary) <- c("classifier", "cluster","cluster_size", "targetCluster",
                                  "alpha","epsilon", "changeClust",
                                  "nbModifiedGenes", "nbTestedGenes",
                                  "oriPourcStart", "targPourcStart",
                                  "oriPourcEnd", "targPourcEnd",
                                   "wholeClust_oriOdd_beg", "wholeClust_targOdd_beg",
                                  "wholeClust_oriOdd_end", "wholeClust_targOdd_end")
    write.table(dfOneRowSummary,
                paste0(baseDir,"oneRowSummary_",expID,".txt"),
                row.names=F)
}

In [8]:
expModifiedExpr <- function(strClassifier="scType",
                            myAttack="cellByCell", modif="perc99",
                            alpha=0.5, epsilon=0.5, myCluster="NK",
                            genesPosition = "bottom", plotSize="large",
                            onlyRedGenes = TRUE, newClassif=NULL, ds="pbmc_3k",
                            ordCells=NULL, show_legend=T, maxSplitSize=200){
    
#     if (is.null(ordCells)){
#         df_pbmc_test <- read.table(c_paths[[ds]]$df)
#         expr_df <- df_pbmc_test[, -which(names(df_pbmc_test) == "y")]
#         clusters_df <- df_pbmc_test$y
#         names(clusters_df) <- rownames(df_pbmc_test)
#         df_pbmc_test_ord <- df_pbmc_test[order(df_pbmc_test$y),]
#         clusters_df_ord <- clusters_df[order(df_pbmc_test$y)]
#     }
    str4key <- strClassifier
    if (strClassifier != "scType" && ds != "pbmc_3k"){
        str4key <- paste0(strClassifier, "_", ds)
    }
    display(str4key)
    myClassifier <- classifList[[str4key]]$fct
    inputType <- classifList[[str4key]]$inputType
    display(inputType)
    mySlot <- classifList[[str4key]]$slot
    expr <- ""    
    oriCT <- "randvalue"
    if(inputType == "data.frame"){
        message("load ", c_paths[[ds]]$df)
        df_pbmc_test <- read.table(c_paths[[ds]]$df)
        expr_df <- df_pbmc_test[, -which(names(df_pbmc_test) == "y")]
        clusters_df <- df_pbmc_test$y
        names(clusters_df) <- rownames(df_pbmc_test)
        df_pbmc_test_ord <- df_pbmc_test[order(df_pbmc_test$y),]
        clusters_df_ord <- clusters_df[order(df_pbmc_test$y)]
        expr <- expr_df
        clusters <- clusters_df
    }
    if(tolower(inputType) == "seurat"){
        message("load ", c_paths[[ds]]$so)
        so_pbmc_test <- readRDS(c_paths[[ds]]$so)
        clusters_so = so_pbmc_test@meta.data$chr_seurat_cluster
        names(clusters_so) <- rownames(so_pbmc_test@meta.data)
        expr <- so_pbmc_test
        clusters <- clusters_so
    }
    if (tolower(inputType) == "singlecellexperiment"){
        message("load ", c_paths[[ds]]$so)
        so_pbmc_test <- readRDS(c_paths[[ds]]$so)
        clusters_so = so_pbmc_test@meta.data$chr_seurat_cluster
        names(clusters_so) <- rownames(so_pbmc_test@meta.data)
        message("load ", c_paths[[ds]]$sce)
        sce_pbmc_test <- readRDS(c_paths[[ds]]$sce)
        clusters_sce = so_pbmc_test@meta.data$chr_seurat_cluster
        names(clusters_sce) <- rownames(so_pbmc_test@meta.data)
        
        expr <- sce_pbmc_test
        clusters <- clusters_sce
    }
    
    expID <- paste0(strClassifier,"_cellByCell_alpha=",alpha,"_epsilon=",epsilon,
                    "__",stringr::str_replace_all(myCluster," ","_"),"__",c_ds4exp[[ds]],"_stopAtSwitch=F_ngene200")
    path_expr <- paste0(bd,"v2/im/temp_data/df_",expID,".txt")
    path_modGenes <- paste0(bd,"v2/im/temp_data/modGenes_",expID,".txt")
    path_oneRow <- paste0(bd,"v2/im/temp_data/oneRowSummary_",expID,".txt")
    path_mct <- paste0(bd,"v2/im/temp_data/modCellTypes_",expID,".txt")
    path_mct_bis <- paste0(bd,"v2/im/temp_data/modCellTypes_",expID,"_bis.txt")
    path_oct <- paste0(bd,"v2/im/temp_data/oriCellTypes_",expID,".txt")
    path_oct_bis <- paste0(bd,"v2/im/temp_data/oriCellTypes_",expID,"_bis.txt")
    display(path_expr)
    if (!file.exists(path_oneRow) || !file.exists(path_mct_bis)){
        display(path_oneRow)
        display(path_expr)
        acbc <- advCGD(expr, clusters, myCluster, myClassifier,
                  genes=lsg[[ds]][1:200],
                     exclNewTargets=NULL, newTarget=NULL,
                     alpha=alpha, epsilon=epsilon,
                     slot=mySlot, verbose=T, stopAtSwitch=F, returnCTpredMat=T)
        display(head(acbc$expr))
        display(path_expr)
        df2export <- acbc$expr
        if (strClassifier == "scAnnotatR"){
            df2export <- as.data.frame(acbc$expr@assays$RNA@layers$scale.data)
            colnames(df2export) <- rownames(acbc$expr@assays$RNA@cells)
            rownames(df2export) <- rownames(acbc$expr@assays$RNA@features)
            df2export <- t(df2export)
        }
#         write.table(df2export, path_expr)
        write.table(acbc$modCellTypesMat, path_mct)
        write.table(acbc$oriCellTypesMat, path_oct)
        write.table(acbc$modGenes, path_modGenes)
        write.table(acbc$oneRowSummary, path_oneRow)
        display("compute pred bis")
        temp_class <- myClassifier(acbc$expr, clusters, myCluster)
        write.table(temp_class$typePredictions, path_mct_bis)

        exportResults(acbc, paste0("data/v2/results/"), expID, strClassifier)
    }
    if (!file.exists(path_oct_bis)){
        temp_class <- myClassifier(expr, clusters, myCluster)
        write.table(temp_class$typePredictions, path_oct_bis)
    }
}

In [9]:
c_args <- c(pbmc_3k="FCGR3A+ Mono",
           axilla_10k="blood vessel endothelial cell",
           liver_10k="macrophage",
           kidney_10k="kidney epithelial cell")
for (str_classif in c("scAnnotatR", "scType")){
    for (my_ds in names(c_args)[4:1]){
        display(str_classif)
        display(my_ds)
        ct <- c_args[my_ds]
        expModifiedExpr(strClassifier=str_classif,
                                alpha=0.5, epsilon=0.5, myCluster=ct,
                                ds=my_ds)
    }
}

[1] "scAnnotatR"

[1] "kidney_10k"

[1] "scAnnotatR_kidney_10k"

[1] "seurat"

load data/v5/data/sc/hgnc_kidney_10k_test.rds



...

load data/v2//pbmc_test.rds



[1] "data/v2/im/temp_data/df_scAnnotatR_cellByCell_alpha=0.5_epsilon=0.5__FCGR3A+_Mono___stopAtSwitch=F_ngene200.txt"

[1] "data/v2/im/temp_data/oneRowSummary_scAnnotatR_cellByCell_alpha=0.5_epsilon=0.5__FCGR3A+_Mono___stopAtSwitch=F_ngene200.txt"

[1] "data/v2/im/temp_data/df_scAnnotatR_cellByCell_alpha=0.5_epsilon=0.5__FCGR3A+_Mono___stopAtSwitch=F_ngene200.txt"

ERROR: Error in subset_models(model_list, cell_types): Error: The following selected classifiers could not be found: malignant cell, blood vessel endothelial cell, fibroblast, macrophage, blood vessel smooth muscle cell, T cell


In [ ]:
c_args <- c(pbmc_3k="FCGR3A+ Mono",
           axilla_10k="blood vessel endothelial cell",
           liver_10k="macrophage",
           kidney_10k="kidney epithelial cell")
for (str_classif in c("scAnnotatR", "scType")){
    for (my_ds in names(c_args)[4:1]){
        display(str_classif)
        display(my_ds)
        ct <- c_args[my_ds]
        expModifiedExpr(strClassifier=str_classif,
                                alpha=1, epsilon=1, myCluster=ct,
                                ds=my_ds)
    }
}

In [14]:
expModifiedExpr <- function(strClassifier="scType",
                            myAttack="cellByCell", modif="perc99",
                            alpha=0.5, epsilon=0.5, myCluster="NK",
                            genesPosition = "bottom", plotSize="large",
                            onlyRedGenes = TRUE, newClassif=NULL, ds="pbmc_3k",
                            ordCells=NULL, show_legend=T, maxSplitSize=200){
    
#     if (is.null(ordCells)){
#         df_pbmc_test <- read.table(c_paths[[ds]]$df)
#         expr_df <- df_pbmc_test[, -which(names(df_pbmc_test) == "y")]
#         clusters_df <- df_pbmc_test$y
#         names(clusters_df) <- rownames(df_pbmc_test)
#         df_pbmc_test_ord <- df_pbmc_test[order(df_pbmc_test$y),]
#         clusters_df_ord <- clusters_df[order(df_pbmc_test$y)]
#     }
    str4key <- strClassifier
    if (strClassifier != "scType" && ds != "pbmc_3k"){
        str4key <- paste0(strClassifier, "_", ds)
    }
    display(str4key)
    myClassifier <- classifList[[str4key]]$fct
    inputType <- classifList[[str4key]]$inputType
    display(inputType)
    mySlot <- classifList[[str4key]]$slot
    expr <- ""    
    oriCT <- "randvalue"
    if(inputType == "data.frame"){
        message("load ", c_paths[[ds]]$df)
        df_pbmc_test <- read.table(c_paths[[ds]]$df)
        expr_df <- df_pbmc_test[, -which(names(df_pbmc_test) == "y")]
        clusters_df <- df_pbmc_test$y
        names(clusters_df) <- rownames(df_pbmc_test)
        df_pbmc_test_ord <- df_pbmc_test[order(df_pbmc_test$y),]
        clusters_df_ord <- clusters_df[order(df_pbmc_test$y)]
        expr <- expr_df
        clusters <- clusters_df
    }
    if(tolower(inputType) == "seurat"){
        message("load ", c_paths[[ds]]$so)
        so_pbmc_test <- readRDS(c_paths[[ds]]$so)
        clusters_so = so_pbmc_test@meta.data$chr_seurat_cluster
        names(clusters_so) <- rownames(so_pbmc_test@meta.data)
        expr <- so_pbmc_test
        clusters <- clusters_so
    }
    if (tolower(inputType) == "singlecellexperiment"){
        message("load ", c_paths[[ds]]$so)
        so_pbmc_test <- readRDS(c_paths[[ds]]$so)
        clusters_so = so_pbmc_test@meta.data$chr_seurat_cluster
        names(clusters_so) <- rownames(so_pbmc_test@meta.data)
        message("load ", c_paths[[ds]]$sce)
        sce_pbmc_test <- readRDS(c_paths[[ds]]$sce)
        clusters_sce = so_pbmc_test@meta.data$chr_seurat_cluster
        names(clusters_sce) <- rownames(so_pbmc_test@meta.data)
        
        expr <- sce_pbmc_test
        clusters <- clusters_sce
    }
    
    expID <- paste0(strClassifier,"_cellByCell_alpha=",alpha,"_epsilon=",epsilon,
                    "__",stringr::str_replace_all(myCluster," ","_"),"__",c_ds4exp[[ds]],"_stopAtSwitch=F_ngene2000")
    path_expr <- paste0(bd,"v2/im/temp_data/df_",expID,".txt")
    path_modGenes <- paste0(bd,"v2/im/temp_data/modGenes_",expID,".txt")
    path_oneRow <- paste0(bd,"v2/im/temp_data/oneRowSummary_",expID,".txt")
    path_mct <- paste0(bd,"v2/im/temp_data/modCellTypes_",expID,".txt")
    path_mct_bis <- paste0(bd,"v2/im/temp_data/modCellTypes_",expID,"_bis.txt")
    path_oct <- paste0(bd,"v2/im/temp_data/oriCellTypes_",expID,".txt")
    path_oct_bis <- paste0(bd,"v2/im/temp_data/oriCellTypes_",expID,"_bis.txt")
    display(path_expr)
    if (!file.exists(path_oneRow) || !file.exists(path_mct_bis)){
        display(path_oneRow)
        display(path_expr)
        acbc <- advCGD(expr, clusters, myCluster, myClassifier,
                  genes=lsg[[ds]][1:2000],
                     exclNewTargets=NULL, newTarget=NULL,
                     alpha=alpha, epsilon=epsilon,
                     slot=mySlot, verbose=T, stopAtSwitch=F, returnCTpredMat=T)
        display(head(acbc$expr))
        display(path_expr)
        df2export <- acbc$expr
        if (strClassifier == "scAnnotatR"){
            df2export <- as.data.frame(acbc$expr@assays$RNA@layers$scale.data)
            colnames(df2export) <- rownames(acbc$expr@assays$RNA@cells)
            rownames(df2export) <- rownames(acbc$expr@assays$RNA@features)
            df2export <- t(df2export)
        }
#         write.table(df2export, path_expr)
        write.table(acbc$modCellTypesMat, path_mct)
        write.table(acbc$oriCellTypesMat, path_oct)
        write.table(acbc$modGenes, path_modGenes)
        write.table(acbc$oneRowSummary, path_oneRow)
        display("compute pred bis")
        temp_class <- myClassifier(acbc$expr, clusters, myCluster)
        write.table(temp_class$typePredictions, path_mct_bis)

        exportResults(acbc, paste0("data/v2/results/"), expID, strClassifier)
    }
    if (!file.exists(path_oct_bis)){
        temp_class <- myClassifier(expr, clusters, myCluster)
        write.table(temp_class$typePredictions, path_oct_bis)
    }
}

In [15]:
c_args <- c(pbmc_3k="FCGR3A+ Mono",
           axilla_10k="blood vessel endothelial cell",
           liver_10k="macrophage",
           kidney_10k="kidney epithelial cell")
for (str_classif in c("RF")){
    for (my_ds in names(c_args)[3]){
        display(str_classif)
        display(my_ds)
        ct <- c_args[my_ds]
        expModifiedExpr(strClassifier=str_classif,
                                alpha=0.5, epsilon=0.5, myCluster=ct,
                                ds=my_ds)
    }
}

[1] "RF"

[1] "liver_10k"

[1] "RF_liver_10k"

[1] "data.frame"

load data/v5/data/sc/hgnc_liver_10k_test.txt



...

 mean 0.603263157894737 delt 0

Number of monocyte : 14

 mean 0.459857142857143 delt 0

Number of modified cells 0

PKP4 559

Number of original annot macrophage : 190

 mean 0.603263157894737 delt 0

Number of monocyte : 14

 mean 0.459857142857143 delt 0

Number of modified cells 65

DOCK8 560

Number of original annot macrophage : 190

 mean 0.603263157894737 delt 0

Number of monocyte : 14

 mean 0.459857142857143 delt 0

Number of modified cells 93

TFPI 561

Number of original annot macrophage : 190

 mean 0.602694736842105 delt -0.000568421052631551

Number of monocyte : 14

 mean 0.459857142857143 delt 0

Number of modified cells 0

CYRIA 562

Number of original annot macrophage : 190

 mean 0.602694736842105 delt 0

Number of monocyte : 14

 mean 0.459857142857143 delt 0

Number of modified cells 5

ENSG00000236283 563

Number of original annot macrophage : 190

 mean 0.602705263157895 delt 1.05263157894608e-05

Number of monocyte : 14

 mean 0.459857142857143 delt 0

Number 

Number of modified cells 135

NOVA1 604

Number of original annot macrophage : 189

 mean 0.601079365079365 delt 0

Number of monocyte : 15

 mean 0.460533333333333 delt 0

Number of modified cells 66

LILRB5 605

Number of original annot macrophage : 189

 mean 0.600994708994709 delt -8.46560846561317e-05

Number of monocyte : 15

 mean 0.460533333333333 delt 0

Number of modified cells 0

UBASH3B 606

Number of original annot macrophage : 189

 mean 0.600994708994709 delt 0

Number of monocyte : 15

 mean 0.460533333333333 delt 0

Number of modified cells 2

ROBO2 607

Number of original annot macrophage : 189

 mean 0.601015873015873 delt 2.11640211640329e-05

Number of monocyte : 15

 mean 0.460533333333333 delt 0

Number of modified cells 6

MIS18BP1 608

Number of original annot macrophage : 189

 mean 0.600984126984127 delt -3.17460317460494e-05

Number of monocyte : 15

 mean 0.460533333333333 delt 0

Number of modified cells 51

BNC2 609

Number of original annot macrophage : 

 mean 0.60007486631016 delt 0

Number of monocyte : 17

 mean 0.451058823529412 delt 0

Number of modified cells 0

HECW2 649

Number of original annot macrophage : 187

 mean 0.60007486631016 delt 0

Number of monocyte : 17

 mean 0.451058823529412 delt 0

Number of modified cells 39

SPARC 650

Number of original annot macrophage : 187

 mean 0.60007486631016 delt 0

Number of monocyte : 17

 mean 0.451058823529412 delt 0

Number of modified cells 53

SNTB1 651

Number of original annot macrophage : 187

 mean 0.600021390374332 delt -5.34759358288284e-05

Number of monocyte : 17

 mean 0.451058823529412 delt 0

Number of modified cells 15

TRIM38 652

Number of original annot macrophage : 187

 mean 0.600021390374332 delt 0

Number of monocyte : 17

 mean 0.451058823529412 delt 0

Number of modified cells 2

SATB2 653

Number of original annot macrophage : 187

 mean 0.600010695187166 delt -1.06951871657879e-05

Number of monocyte : 17

 mean 0.451058823529412 delt 0

Number of modif

 mean 0.451058823529412 delt 0

Number of modified cells 0

RNF213 694

Number of original annot macrophage : 187

 mean 0.596844919786096 delt 0

Number of monocyte : 17

 mean 0.451058823529412 delt 0

Number of modified cells 109

KLRB1 695

Number of original annot macrophage : 187

 mean 0.595283422459893 delt -0.00156149732620314

Number of monocyte : 17

 mean 0.451058823529412 delt 0

Number of modified cells 0

ENSG00000254420 696

Number of original annot macrophage : 187

 mean 0.595283422459893 delt 0

Number of monocyte : 17

 mean 0.451058823529412 delt 0

Number of modified cells 60

RBMS3 697

Number of original annot macrophage : 187

 mean 0.595208556149733 delt -7.48663101604041e-05

Number of monocyte : 17

 mean 0.451058823529412 delt 0

Number of modified cells 58

HLA.DMB 698

Number of original annot macrophage : 187

 mean 0.595208556149733 delt 0

Number of monocyte : 17

 mean 0.451058823529412 delt 0

Number of modified cells 19

ATP2B4 699

Number of origin

MBNL1 739

Number of original annot macrophage : 187

 mean 0.59192513368984 delt -0.000106951871657768

Number of monocyte : 17

 mean 0.451058823529412 delt 0

Number of modified cells 88

DOCK5 740

Number of original annot macrophage : 187

 mean 0.59103743315508 delt -0.000887700534759395

Number of monocyte : 17

 mean 0.451058823529412 delt 0

Number of modified cells 25

LMOD1 741

Number of original annot macrophage : 187

 mean 0.590973262032086 delt -6.41711229946162e-05

Number of monocyte : 17

 mean 0.451058823529412 delt 0

Number of modified cells 0

RAB39A 742

Number of original annot macrophage : 187

 mean 0.590973262032086 delt 0

Number of monocyte : 17

 mean 0.451058823529412 delt 0

Number of modified cells 0

ABCC3 743

Number of original annot macrophage : 187

 mean 0.590973262032086 delt 0

Number of monocyte : 17

 mean 0.451058823529412 delt 0

Number of modified cells 2

CD36 744

Number of original annot macrophage : 187

 mean 0.590951871657754 delt -2

In [12]:
c_args <- c(pbmc_3k="FCGR3A+ Mono",
           axilla_10k="blood vessel endothelial cell",
           liver_10k="macrophage",
           kidney_10k="kidney epithelial cell")
for (str_classif in c("scAnnotatR", "scType")){
    for (my_ds in names(c_args)[4:1]){
        display(str_classif)
        display(my_ds)
        ct <- c_args[my_ds]
        expModifiedExpr(strClassifier=str_classif,
                                alpha=1, epsilon=1, myCluster=ct,
                                ds=my_ds)
    }
}

[1] "scAnnotatR"

[1] "kidney_10k"

[1] "scAnnotatR_kidney_10k"

[1] "seurat"

load data/v5/data/sc/hgnc_kidney_10k_test.rds



...

load data/v2//pbmc_test.rds



[1] "data/v2/im/temp_data/df_scAnnotatR_cellByCell_alpha=1_epsilon=1__FCGR3A+_Mono___stopAtSwitch=F_ngene50.txt"

[1] "data/v2/im/temp_data/oneRowSummary_scAnnotatR_cellByCell_alpha=1_epsilon=1__FCGR3A+_Mono___stopAtSwitch=F_ngene50.txt"

[1] "data/v2/im/temp_data/df_scAnnotatR_cellByCell_alpha=1_epsilon=1__FCGR3A+_Mono___stopAtSwitch=F_ngene50.txt"

ERROR: Error in subset_models(model_list, cell_types): Error: The following selected classifiers could not be found: malignant cell, blood vessel endothelial cell, fibroblast, macrophage, blood vessel smooth muscle cell, T cell
